In [2]:
import boto3
import pandas as pd
import json
import matplotlib.pyplot as plt
import time
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForTokenClassification
import os
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

## load questions and sql queries in spider train

In [3]:
# Define your S3 bucket and file key
bucket_name = 'sagemaker-studio-423623869859-3no3d9ie4hx'
file_key = 'train_spider.json'  # replace with your actual file path

# Initialize the S3 client
s3_client = boto3.client('s3')

# Load the file from S3
obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)
json_train = json.load(obj['Body'])

# Load questions and answers into a pandas dataframe
pd.set_option('display.max_colwidth', None)

list_questions_and_sqls = []

for ele in json_train:
    list_questions_and_sqls.append([ele['question'], ele['query']])
    
df_questions_and_sqls = pd.DataFrame(list_questions_and_sqls, columns=['question', 'sql_query'])

In [4]:
df_questions_and_sqls.head(5)

,question,sql_query
0,How many heads of the departments are older than 56 ?,SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads of departments ordered by age.","SELECT name , born_state , age FROM head ORDER BY age"
2,"List the creation year, name and budget of each department.","SELECT creation , name , budget_in_billions FROM department"
3,What are the maximum and minimum budget of the departments?,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department"
4,What is the average number of employees of the departments whose rank is between 10 and 15?,SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15


## sentence embedding with BERT

In [6]:
# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

In [8]:
# Example sentences to generate embeddings for
sentences = [
    "Artificial intelligence is transforming healthcare.",
    "AI is making significant advances in medical diagnostics.",
    "Natural language processing helps computers understand human language."
]

# Generate embeddings
embeddings = model.encode(sentences)

# Display the embeddings
for i, sentence in enumerate(sentences):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embeddings[i][:5]}...")  # Displaying only the first 5 values for brevity
    print(len(embeddings[i]))
    print()

Sentence: Artificial intelligence is transforming healthcare.
Embedding: [-0.03841847 -0.01910543 -0.04153921  0.00217933 -0.05643337]...
384

Sentence: AI is making significant advances in medical diagnostics.
Embedding: [-0.05644108 -0.06398027 -0.00725187 -0.01021199  0.00754326]...
384

Sentence: Natural language processing helps computers understand human language.
Embedding: [ 0.00287584 -0.00609016  0.03169986  0.0238028   0.00913628]...
384

